https://www.kaggle.com/datasets/alphiree/cardiovascular-diseases-risk-prediction-dataset
Clasificacion Multiclase:Se utiliza para categorizar datos en mas de dos clases,es eficiente a la hora de entrenarun solo modelo multiclase, lo que es mas eficiente computacionalmente hablando en terminos de memoria, y lo malo es que mientras mas complejo el modelo se necesitara mas recurso computacional, y es ligeramente menos preciso que el binario, cuando hay clases desbalansadas.

 Se lo puede utilizar en diagnosticos medicos como aremos en siguiente ejemplo utilizando el dataset del encabezado

In [ ]:
# utilizado para manejos de directorios y rutas
import os
#impotamos la libreria de pandas para el manejo de los datos:
import pandas as pd
# Computacion vectorial y cientifica para python
import numpy as np

# Librerias para graficación (trazado de gráficos)
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D  # Necesario para graficar superficies 3D
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
from scipy.io import loadmat

Primero abrimos el archivo  csv del dataset(CVD_cleaned.csv)

In [ ]:
datos =pd.read_csv('CVD_cleaned.csv', delimiter=',')
datos

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,Poor,Within the past 2 years,No,No,No,No,No,No,Yes,Female,70-74,150.0,32.66,14.54,Yes,0.0,30.0,16.0,12.0
1,Very Good,Within the past year,No,Yes,No,No,No,Yes,No,Female,70-74,165.0,77.11,28.29,No,0.0,30.0,0.0,4.0
2,Very Good,Within the past year,Yes,No,No,No,No,Yes,No,Female,60-64,163.0,88.45,33.47,No,4.0,12.0,3.0,16.0
3,Poor,Within the past year,Yes,Yes,No,No,No,Yes,No,Male,75-79,180.0,93.44,28.73,No,0.0,30.0,30.0,8.0
4,Good,Within the past year,No,No,No,No,No,No,No,Male,80+,191.0,88.45,24.37,Yes,0.0,8.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308849,Very Good,Within the past year,Yes,No,No,No,No,No,No,Male,25-29,168.0,81.65,29.05,No,4.0,30.0,8.0,0.0
308850,Fair,Within the past 5 years,Yes,No,No,No,No,Yes,No,Male,65-69,180.0,69.85,21.48,No,8.0,15.0,60.0,4.0
308851,Very Good,5 or more years ago,Yes,No,No,No,Yes,"Yes, but female told only during pregnancy",No,Female,30-34,157.0,61.23,24.69,Yes,4.0,40.0,8.0,4.0
308852,Very Good,Within the past year,Yes,No,No,No,No,No,No,Male,65-69,183.0,79.38,23.73,No,3.0,30.0,12.0,0.0


vemos de que tipo de son las comunas para poder adecuarlas

In [ ]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308854 entries, 0 to 308853
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   General_Health                308854 non-null  object 
 1   Checkup                       308854 non-null  object 
 2   Exercise                      308854 non-null  object 
 3   Heart_Disease                 308854 non-null  object 
 4   Skin_Cancer                   308854 non-null  object 
 5   Other_Cancer                  308854 non-null  object 
 6   Depression                    308854 non-null  object 
 7   Diabetes                      308854 non-null  object 
 8   Arthritis                     308854 non-null  object 
 9   Sex                           308854 non-null  object 
 10  Age_Category                  308854 non-null  object 
 11  Height_(cm)                   308854 non-null  float64
 12  Weight_(kg)                   308854 non-nul

categorizamos las columnas de tipo object para hacer los calculos numericos

In [ ]:
columnas_categoricas = datos.select_dtypes(include=['object']).columns

In [ ]:
#Procesamiento de frases
for columna in columnas_categoricas:
  le = LabelEncoder()
  datos[columna] = le.fit_transform(datos[columna])

In [ ]:
datos

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,3,2,0,0,0,0,0,0,1,0,10,150.0,32.66,14.54,1,0.0,30.0,16.0,12.0
1,4,4,0,1,0,0,0,2,0,0,10,165.0,77.11,28.29,0,0.0,30.0,0.0,4.0
2,4,4,1,0,0,0,0,2,0,0,8,163.0,88.45,33.47,0,4.0,12.0,3.0,16.0
3,3,4,1,1,0,0,0,2,0,1,11,180.0,93.44,28.73,0,0.0,30.0,30.0,8.0
4,2,4,0,0,0,0,0,0,0,1,12,191.0,88.45,24.37,1,0.0,8.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308849,4,4,1,0,0,0,0,0,0,1,1,168.0,81.65,29.05,0,4.0,30.0,8.0,0.0
308850,1,3,1,0,0,0,0,2,0,1,9,180.0,69.85,21.48,0,8.0,15.0,60.0,4.0
308851,4,0,1,0,0,0,1,3,0,0,2,157.0,61.23,24.69,1,4.0,40.0,8.0,4.0
308852,4,4,1,0,0,0,0,0,0,1,9,183.0,79.38,23.73,0,3.0,30.0,12.0,0.0


Identificamos las calses (y) que y las caracteristicas de cada clase(X)

In [ ]:
X = datos.iloc[:, 1:]
y = datos.iloc[:, 0]
m = y.size
num_labels = 5
print(X.shape)
print(y.shape)
print(X)
print('---'*20)
print(y)

(308854, 18)
(308854,)
        Checkup  Exercise  Heart_Disease  Skin_Cancer  Other_Cancer  \
0             2         0              0            0             0   
1             4         0              1            0             0   
2             4         1              0            0             0   
3             4         1              1            0             0   
4             4         0              0            0             0   
...         ...       ...            ...          ...           ...   
308849        4         1              0            0             0   
308850        3         1              0            0             0   
308851        0         1              0            0             0   
308852        4         1              0            0             0   
308853        4         1              0            0             0   

        Depression  Diabetes  Arthritis  Sex  Age_Category  Height_(cm)  \
0                0         0          1    0     

podemos observar que tenemos 5 clases en nuestro dataset:
0.-exelente
1.-regular
2.-bueno
3.-mala
4.-muy buena

In [ ]:
etiquetas_unicas = np.unique(y)
cantidad_etiquetas = len(etiquetas_unicas)

print("Etiquetas únicas:", etiquetas_unicas)
print("Cantidad de etiquetas:", cantidad_etiquetas)

Etiquetas únicas: [0 1 2 3 4]
Cantidad de etiquetas: 5


luego procedemos a normalizar para poder manejar los datos  entre 0 y 1

In [ ]:
X_mean, X_std = X.mean(axis=0), X.std(axis=0) #X.mean Se calcula la media de cada columna a lo largo de las filas, y X.std la desciacion estandar de cada columna
X_norm = (X - X_mean) / X_std #se calcula la media restando la media de cada columna y dividiendo por su desviacion

X.shape, y.shape

((308854, 18), (308854,))

Luego definimos la funion de la sigmoide: toma un valor real como entrada y lo devuelve en un rango de 0 y 1

In [ ]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [ ]:
def lrCostFunction(theta, X, y, lambda_):
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [ ]:
def oneVsAll(X, y, num_labels, lambda_):

    # algunas variables utiles
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [ ]:
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)

<ipython-input-113-58a428056341>:16: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-113-58a428056341>:16: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))


In [ ]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [ ]:
predicciones = predictOneVsAll(all_theta, X_norm)
precision = np.mean(predicciones == y) * 100
print('Precisión en el conjunto de entrenamiento: {:.2f}%'.format(precision))

Precisión en el conjunto de entrenamiento: 30.70%
